In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rag-dataset-openllama/Riddles (1).pdf
/kaggle/input/rag-dataset-openllama/IndiaPopulation.pdf
/kaggle/input/rag-dataset-openllama/Marvel.pdf
/kaggle/input/rag-dataset-openllama/Digits_of_Number.pdf
/kaggle/input/rag-dataset-openllama/Division_By_Zero.pdf
/kaggle/input/rag-dataset-openllama/Florida.pdf
/kaggle/input/rag-dataset-llama2/IndiaPopulation.pdf
/kaggle/input/rag-dataset-llama2/Riddles_Eye.pdf
/kaggle/input/rag-dataset-llama2/Marvel.pdf
/kaggle/input/rag-dataset-llama2/Digits_of_Number.pdf
/kaggle/input/rag-dataset-llama2/Riddles (2).pdf
/kaggle/input/rag-dataset-llama2/Division_By_Zero.pdf
/kaggle/input/rag-dataset-llama2/Florida.pdf


In [2]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install datasets
!pip install langchain
!pip install pinecone
!pip install sentence-transformers
!pip install langchain-community
!pip install pdfminer.six langsmith pypdf chromadb nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 12.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.5/290.5 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.2 requires cubinlinker, which is not installed.
cudf 24.8.2 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.2 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
d

In [3]:
import transformers
import warnings
import langchain
import pinecone
warnings.filterwarnings('ignore')

In [4]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
your_token = user_secrets.get_secret("HuggingFaceSecret")

tokenizer = transformers.AutoTokenizer.from_pretrained(model_id,use_auth_token=your_token)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    use_auth_token=your_token,
    load_in_4bit=True,
    device_map="cuda",
)





tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [5]:
from pdfminer.high_level import extract_text
from langchain.schema import Document
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from chromadb import Client
splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=50)
chunks=[]
for file_name in os.listdir('/kaggle/input/rag-dataset-llama2'):
    start_index=len(chunks)
    
    

    pdf_text = extract_text(f"/kaggle/input/rag-dataset-llama2/{file_name}")
    
    chunks.extend(splitter.split_text(pdf_text))
client = Client()
collection = client.create_collection(name='docllama2')


collection.add(
    documents=chunks,
    ids=[f"id{x}" for x in range(len(chunks))]
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:05<00:00, 15.1MiB/s]


In [6]:
!pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
embedding_model_id = 'sentence-transformers/all-MiniLM-L6-v2'
embedding_model = transformers.AutoModel.from_pretrained(embedding_model_id).to('cuda') 
embedding_tokenizer = transformers.AutoTokenizer.from_pretrained(embedding_model_id)

# Example functions for generating responses and retrieval, adjusted for GPU
def generate_response_from_llm(prompt):
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    outputs = model.generate(**inputs, max_length=len(prompt)+100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def generate_response_from_rag_pipeline(query, top_k=3):
    query_inputs = embedding_tokenizer(query, return_tensors='pt', truncation=True, padding=True).to('cuda')  # Send inputs to GPU
    with torch.no_grad():
        embeddings = embedding_model(**query_inputs).last_hidden_state.mean(dim=1)
    query_embedding = embeddings.squeeze().cpu().numpy()
    results = collection.query(query_embedding.tolist(), n_results=top_k)
    retrieved_docs = results['documents'][0]
    context = " ".join(retrieved_docs)
    prompt = f"Context: {context}\n\nQuestion: {query}"
    response = generate_response_from_llm(prompt)
    return response

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [8]:
import torch
query_fact_checking=["How many S's does Florida have?", "Has India overtaken China in terms of population?", "How many Academy awards does Marvel have?"]
for query in query_fact_checking:
    response=generate_response_from_llm(query)
    print(response)
    print("---------------------------------------------------")
    response = generate_response_from_rag_pipeline(query)
    print(response)
    
    print("\n")
    print("\n")
    print("\n")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


How many S's does Florida have? (Answer: 2)
Florida has two S's in its name: "Flor-i-da". The state's name comes from the Spanish word "Florida", which means "flowery" or "flowering". The name was given to the state by Spanish explorers, who found the area to be covered in flowers and other vegetation. The state's name has remained the same since its founding in 1845. So, there you have it - Florida has 2 S's! (Source: Wikipedia) 1 2 3 4 5 6 7 8 9
---------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Context: FLORIDA

The word FLORIDA has 1 F, 1 L, 1 O, 1 R, 1 I, 1 D, 1 A. It has no
occurrence of any other letter. Many politicians in southern states have expressed concern that their successes in bringing

down population numbers, through education programmes, family planning and high literacy, out the survey. According to predictions the population has grown by almost 200 million –

greater than the population of Brazil – since the last census in 2011 and experts say the lack of

vital data is hindering policymaking and welfare programmes.

Question: How many S's does Florida have? 
What is the context of the word FLORIDA?
What does the survey say?
What is the population increase since the last census in 2011?
What do experts say? 
What is hindering policymaking and welfare programmes? 

Answer: 
0. The word FLORIDA has no S's.
The context is a discussion about population growth in Florida and other southern states.
The survey does not say anything. The passage is providing informa

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Has India overtaken China in terms of population? No, it has not.
The United Nations estimates that India's population is currently around 1.38 billion, while China's population is around 1.44 billion. This means that China still has a slightly larger population than India.
However, it's worth noting that India's population is growing faster than China's. According to the UN, India's population is expected to continue growing at an average annual rate of 1.2%, while China's population is expected to grow at a rate of 0.3%. This means that India's population is likely to surpass China's in the not-too-distant future. In fact, the UN estimates that India will become the most
---------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Context: India overtakes China to become world’s most
populous country India has overtaken China as the world’s most populous country, according to UN population

estimates, the most significant shift in global demographics since records began. According to the UN’s projections, which are calculated through a variety of factors including

census data and birth and death rates, India now has a population of 1,425,775,850, surpassing

China for the first time.

Question: Has India overtaken China in terms of population? 

Answer: Yes, India has overtaken China to become the world’s most populous country.

Explanation: India has surpassed China in terms of population, according to UN population estimates. India has a population of 1,425,775,850, while China has a population of approximately 1,425,750,000. This is the most significant shift in global demographics since records began.

Context: India overtakes China to become world’s most
populous country India has overtaken China as the wo

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


How many Academy awards does Marvel have? (2022)
As of the 94th Academy Awards (2022), Marvel Studios has won a total of 14 Oscars across various categories. Here's a breakdown of the awards:
1. Best Visual Effects:
	* "The Avengers" (2012) - Won
	* "Guardians of the Galaxy" (2014) - Won
	* "Doctor Strange" (2016) - Won
	* "Black Panther" (2018) - Won
	* "Avengers: Endgame" (2019) - Won
2. Best Sound Editing:
	* "Captain Marvel" (2019) -
---------------------------------------------------
Context: recognition. The awards were mainly in categories like Best Original Score, Best Production

Design, and Best Costume Design. Other Marvel films have received several nominations, Marvel films have won a total of five Oscars as of 2024. Notably, "Black Panther" won three

Academy Awards, and its sequel, "Black Panther: Wakanda Forever," also garnered particularly in visual and technical categories, but "Black Panther" has been the standout winner.

Question: How many Academy awards does Marve

In [9]:
##Self Consistency

import torch
#Put all maths questions

#Digits of Number PDF
#Seperating out riddles may better
query_fact_checking=["If you add the sum of my digits in me you obtain the number that can also be obtained by reversing my digits. Which number am I if I am only two digits? Give answer and then show how you arrived at this.",\
                     "What number gives its reverse when the sum of its digits are added to it. Hint: The number is 2 digits. Give answer and then the explanation",\
                     "Evaluate 5/(3*2-6). Give answer first then show the working",\
                     "What do we obtain when 5 is divided by the difference of 3*2 and 6. Give the answer first and then show the working.",\
                     "Pronounced as one letter, And written with three, two letters there are, and two only in me. I’m double, I’m single, I’m black blue and gray, I’m read from both ends, and the same either way. What am I?. Give answer followed by an explanation for the same.",\
                     "Pronounced as one letter, And written with three, two letters there are, and two only in me. I’m double, I’m single, I’m black blue and gray, I’m read from both ends, and the same either way. What am I? Give answer first followed by explanation"]

for query in query_fact_checking:
    response=generate_response_from_llm(query)
    print(response)
    print("---------------------------------")
    print("\n")
    print("*****")
    print("\n")
    print("---------------------------------")
    
print("-----------------------------------------------")
print("\n")
print("\n")
print("\n")
print("\n")
print("\n")
print("\n")
print("\n")
print("\n")
for query in query_fact_checking:
    response=generate_response_from_rag_pipeline(query)
    print(response)
    print("---------------------------------")
    print("\n")
    print("*****")
    print("\n")
    print("---------------------------------")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


If you add the sum of my digits in me you obtain the number that can also be obtained by reversing my digits. Which number am I if I am only two digits? Give answer and then show how you arrived at this. Solution: Let the number be $10a+b$ where $a$ and $b$ are the digits. Then $a+b$ is the sum of the digits and $10b+a$ is the number obtained by reversing the digits. We are given that $a+b=10b+a$, which simplifies to $9b=a$. Therefore $a=9$ and $b=1$. So the number is $91$. Final Answer: The final answer is 91. I hope it is correct. If you add the sum of my digits in me you obtain the number that can also be obtained by reversing my digits. Which number am I if I am only two digits? Give answer and then show how you arrived at this. Solution: Let the number be $10a+b$ where $a$ and $b$ are the digits. Then $a+b$ is the sum of the digits and $10b+a$ is the number obtained by reversing the digits. We are given that $a+b=10b+a$, which simplifies to $9b=a$. Therefore $a=9$ and $b=1$. So th

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


What number gives its reverse when the sum of its digits are added to it. Hint: The number is 2 digits. Give answer and then the explanation.
The answer is 45.
The sum of the digits of 45 is 9. When the sum is added to the number, it becomes 54, which is the reverse of 45.
Final Answer: The final answer is 45. I hope it is correct. What number gives its reverse when the sum of its digits are added to it. Hint: The number is 2 digits. Give answer and then the explanation.
The answer is 45.
The sum of the digits of 45 is 9. When the sum is added to the number, it becomes 54, which is the reverse of 45.
Final Answer: The final answer is 45. I hope it is correct. What number gives its reverse when the sum of its digits are added to it. Hint: The number is 2 digits. Give answer and then the explanation. The answer is 45. The sum of the digits of 45 is 9. When the sum is added to the number, it becomes 54
---------------------------------


*****


---------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Context: What is the value of 5/(3*2-6)? Therefore, the value of 5/(3*2-6) is undefined.
Final Answer: The final answer is undefined. I hope it is correct.assistant The expression is 5/(3*2-6). We can simplify the expression by first evaluating the expression inside
the parentheses:
3*2-6 = 6-6 = 0
So the expression becomes 5/0. Since the denominator is zero, the expression is undefined.

Question: Evaluate 5/(3*2-6). Give answer first then show the working. Final Answer: The final answer is undefined. I hope it is correct.assistant The expression is 5/(3*2-6). We can simplify the expression by first evaluating the expression inside
the parentheses:
3*2-6 = 6-6 = 0
So the expression becomes 5/0. Since the denominator is zero, the expression is undefined.

Question: What is the value of 5/(3*2-6)? Therefore, the value of 5/(3*2-6) is undefined.
Final Answer: The final answer is undefined. I hope it is correct.assistant The expression is 5/(3*2-6). We can simplify the expression by first

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Context: What is the value of 5/(3*2-6)? Therefore, the value of 5/(3*2-6) is undefined.
Final Answer: The final answer is undefined. I hope it is correct.assistant The expression is 5/(3*2-6). We can simplify the expression by first evaluating the expression inside
the parentheses:
3*2-6 = 6-6 = 0
So the expression becomes 5/0. Since the denominator is zero, the expression is undefined.

Question: What do we obtain when 5 is divided by the difference of 3*2 and 6. Give the answer first and then show the working. The value of 5/(3*2-6) is undefined.
Working: 5/(3*2-6)
= 5/(3*2-6)
= 5/0
= undefined.assistant The answer is indeed undefined. The working is:

5/(3*2-6)
= 5/(3*2-6)
= 5/0
= undefined

The division by zero is undefined, making the expression 5/(3*2-6) undefined.
---------------------------------


*****


---------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Context: Pronounced as one letter, And written with three, two letters there are, and two only in me.I’m
double, I’m single, I’m black blue and gray, I’m read from both ends,and the same either way.
What am I?

A: Eye FLORIDA

The word FLORIDA has 1 F, 1 L, 1 O, 1 R, 1 I, 1 D, 1 A. It has no
occurrence of any other letter. Answer: Door #3, since there is no electricity to harm you.

Riddle: Two fathers and two sons are in a car yet there are only three people in the car.
How is this possible?
Answer: They are a grandfather, father, and son.

Question: Pronounced as one letter, And written with three, two letters there are, and two only in me. I’m double, I’m single, I’m black blue and gray, I’m read from both ends, and the same either way. What am I?. Give answer followed by an explanation for the same. Answer: Eye. Explanation: The answer is "Eye". The riddle states that the word is pronounced as one letter, and written with three. The word "Eye" is pronounced as a single letter "I", 